In [1]:
import json
import numpy as np
from pathlib import Path

In [2]:
def alpha_schedule(t, T):
    """t=0..T-1. Example: linearly increases stability weight over time."""
    return 0.3 + 0.7 * (t / max(T-1, 1))     # starts 0.3, ends 1.0



def utility(P, S, E, lamda, wP, wS, wE):
    """
    P,S,E in [0,1]. Weights nonnegative. Uses natural logs.
    U = [ Π (1+λ x)^{w_x / ln(1+λ)}  - 1 ] / (e^{W} - 1), W=wP+wS+wE
    """
    L = np.log(1.0 + lamda)  # ln(1+λ)
    W = wP + wS + wE
    # product term (vectorized)
    prod = ((1.0 + lamda*P) ** (wP / L) *
            (1.0 + lamda*S) ** (wS / L) *
            (1.0 + lamda*E) ** (wE / L))
    num = prod - 1.0
    den = np.exp(W) - 1.0
    return num / den

In [13]:
results_dir = Path("/proj/sourasb-220503/IoT_attack_CL_IDS/results")
lam = 20.0
# alpha_default = 0.6
USE_ALPHA_SCHEDULE = False
prepend_zero = True   # since you wanted FWT[0]=0 and BWT[0]=0


for file_path in results_dir.glob("1_experiment_results_LSTM_LwF*.json"):
    print(f"Processing {file_path.name} ...")

    with open(file_path, "r") as f:
        data = json.load(f)
    

    cost_dict  = data["domain_training_cost"]
    FWT_values = np.asarray(data["FWT_values"], dtype=float)  # length 48
    FWT_values = np.insert(FWT_values, 0, 0.0)  # FWT for first domain is always 0.0
    BWT_values = np.asarray(data["BWT_values"], dtype=float)  # length 48
    BWT_values = np.insert(BWT_values, 0, 0.0)  # BWT for first domain is always 0.0
    order      = data["train_domain_order"]                   # length 48

    T = len(order)
    # print(f" order: {T}",)
    # print(f"Computing utility for {T} domains...")
    # print(f" - FWT  length: {len(FWT_values)}")
    # print(f" - BWT  length: {len(BWT_values)}")    
    # Check lengths match   
    if not (len(FWT_values) == len(BWT_values) == T):
        raise ValueError("Length mismatch: FWT_values, BWT_values, and train_domain_order must match.")

    # -----------------------------
    # Build cost array in the given order
    # cost_dict values are lists like: "name": [ value ]
    # -----------------------------
    cost_list = []
    for name in order:
        if name not in cost_dict:
            raise KeyError(f"Domain '{name}' not found in 'domain_training_cost'.")
        val_list = cost_dict[name]
        if not isinstance(val_list, list) or len(val_list) == 0:
            raise ValueError(f"Cost for domain '{name}' should be a non-empty list.")
        cost_list.append(float(val_list[0]))
    cost = np.asarray(cost_list, dtype=float)

    # -----------------------------
    # Map to P, S, E in [0,1]
    # -----------------------------
    # Plasticity: FWT may be in [-1,1] -> map to [0,1]
    P = (FWT_values + 1.0) / 2.0

    # Stability: BWT in [-1,1] -> map to [0,1]
    S = (BWT_values + 1.0) / 2.0

    # Efficiency: from cost -> normalize to [0,1], then E = 1 - norm_cost
    c_min, c_max = float(np.min(cost)), float(np.max(cost))
    if c_max > c_min:
        cost_norm = (cost - c_min) / (c_max - c_min)
    else:
        # All costs equal -> treat all as 0 (best) after normalization
        cost_norm = np.zeros_like(cost)
    E = 1.0 - cost_norm

    # Clip for numerical safety
    P = np.clip(P, 0.0, 1.0)
    S = np.clip(S, 0.0, 1.0)
    E = np.clip(E, 0.0, 1.0)

    # -----------------------------
    # Weights per step (α_t)
    # -----------------------------
    USE_ALPHA_SCHEDULE = False  # set True to use the schedule above

    # if USE_ALPHA_SCHEDULE:
    #     alpha_t = np.array([alpha_schedule(t, T) for t in range(T)], dtype=float)
    # else:
    #     alpha_t = np.full(T, alpha_default, dtype=float)

    # print(f"Using alpha_t: {alpha_t}")

    wP = np.full(T, 2.0, dtype=float)
    wS = np.full(T, 0.0, dtype=float) #alpha_t
    wE = np.full(T, 0.0, dtype=float)#1.0 - alpha_t

    # print(f"wP: {wP}")
    # print(f"wS: {wS}")
    # print(f"wE: {wE}")


    # -----------------------------
    # Compute Utility for all 48
    # -----------------------------
    U = utility(P, S, E, lamda=lam, wP=wP, wS=wS, wE=wE).tolist()

    # -----------------------------
    # Save back to JSON
    # -----------------------------
    data["Utility_WP_2"] = U
    data["Utility_params"] = {
        "lambda": lam,
        # "alpha_schedule": "linear_0.3_to_1.0" if USE_ALPHA_SCHEDULE else f"constant_{alpha_default}",
        "weights_note": f"w_P={wP}, w_S={wS}, w_E= {wE}",
        "normalization": {
            "P": "P = (FWT+1)/2",
            "S": "S = (BWT+1)/2",
            "E": "E = 1 - minmax(cost)"
        }
    }

    print(data["Utility_params"])
    print(data["Utility_WP_2"])
    with open(file_path, "w") as f:
         json.dump(data, f, indent=2)

    print(f"Done. Wrote utilities to {file_path}")


Processing 1_experiment_results_LSTM_LwF_b2w_alpha_1.0_T_4.0.json ...
{'lambda': 20.0, 'weights_note': 'w_P=[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.\n 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], w_S=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], w_E= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]', 'normalization': {'P': 'P = (FWT+1)/2', 'S': 'S = (BWT+1)/2', 'E': 'E = 1 - minmax(cost)'}}
[0.5997444886839577, 0.5944561095817904, 0.5769850923303428, 0.5722996276028821, 0.5792108243204533, 0.5833413052993678, 0.5860882796106517, 0.5880471589364241, 0.5895145537394315, 0.5906106732474754, 0.5915002013968691, 0.5922516382460639, 0.58296900954632, 0.5842355032676696, 0.5852756562363131, 0.5861167428869773, 0.5858344291131364, 0.58